In [ ]:
import cv2
import numpy as np
import os

# Function to hide message using a spatial domain technique
def hide_message_spatial(image, binary_message):
    # Flatten image pixel values
    flattened_image = image.reshape((-1, 3))

    # Process 4 bits at a time
    for i in range(0, len(binary_message), 4):
        # Get the next 4 bits of the message
        bits = binary_message[i:i + 4]
        bits = bits.ljust(4, '0')  # Pad with zeros if less than 4 bits

        # Convert bits to integer
        bits_int = int(bits, 2)

        # Embed 4 bits by modifying the pixel intensity value
        flattened_image[i // 4][0] = (flattened_image[i // 4][0] & 240) | bits_int
    
    # Reshape the image back to original shape
    modified_image = flattened_image.reshape(image.shape)

    return modified_image

# Function to extract message hidden using the spatial domain technique
def extract_message_spatial(image, message_length):
    # Flatten image pixel values
    flattened_image = image.reshape((-1, 3))

    # Extract 4 bits from each pixel
    extracted_message = []
    for i in range(message_length // 4):
        bits_int = flattened_image[i][0] & 15
        bits = format(bits_int, '04b')
        extracted_message.append(bits)
    
    # Join the bits into a single binary string
    binary_message = ''.join(extracted_message)

    return binary_message

# Function to write binary message to a text file
def write_binary_message_to_file(binary_message, file_path):
    with open(file_path, 'w') as file:
        file.write(binary_message)

# Function to calculate SNR between two images
def calculate_snr(original_image, modified_image):
    # Convert images to float64 to avoid overflow during computations
    original_image = original_image.astype(np.float64)
    modified_image = modified_image.astype(np.float64)
    
    # Calculate signal power
    signal_power = np.sum(original_image ** 2)
    
    # Calculate noise power as the sum of squared differences
    noise = original_image - modified_image
    noise_power = np.sum(noise ** 2)
    
    # Calculate SNR in decibels (dB)
    snr = 10 * np.log10(signal_power / noise_power)
    
    return snr

# Function to calculate maximum embedding capacity of the cover image
def calculate_max_capacity(image):
    height, width, channels = image.shape
    
    # Calculate the total number of bits that can be hidden
    max_capacity_bits = height * width * channels * 4
    
    return max_capacity_bits

# Paths for input files and directories
binary_message_path = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\random_bits.txt'
cover_image_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\cover_image'
output_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00'

# Read binary data from text file
with open(binary_message_path, 'r') as file:
    binary_message = file.read()

# Determine the number of cover images needed
cover_image_files = sorted([os.path.join(cover_image_dir, f) for f in os.listdir(cover_image_dir) if f.endswith('.jpg')])
num_cover_images = len(cover_image_files)

# Load the first cover image to get its dimensions
cover_image = cv2.imread(cover_image_files[0])
cover_image_rgb = cv2.cvtColor(cover_image, cv2.COLOR_BGR2RGB)
max_capacity = calculate_max_capacity(cover_image_rgb)

# Split binary message into chunks
chunk_size = max_capacity // 4
binary_message_chunks = [binary_message[i:i + chunk_size] for i in range(0, len(binary_message), chunk_size)]
num_full_chunks = len(binary_message_chunks)

# Embed each chunk into a separate cover image
modified_images = []
for j, chunk in enumerate(binary_message_chunks):
    # Determine the current cover image index using modulo operation
    i = j % num_cover_images
    
    # Load the current cover image
    cover_image = cv2.imread(cover_image_files[i])
    cover_image_rgb = cv2.cvtColor(cover_image, cv2.COLOR_BGR2RGB)
    
    # Embed the message chunk into the current cover image
    modified_image = hide_message_spatial(cover_image_rgb, chunk)
    modified_images.append(modified_image)
    
    # Save the modified image with a sequential name
    output_image_path = os.path.join(output_dir, f'{j + 1}.jpg')
    cv2.imwrite(output_image_path, cv2.cvtColor(modified_image, cv2.COLOR_RGB2BGR))
    print(f"Modified image {j + 1} saved to: {output_image_path}")

# Extract hidden message from each modified image
extracted_messages = []
for j, modified_image in enumerate(modified_images):
    # Determine the current cover image index using modulo operation
    i = j % num_cover_images
    
    chunk_size = len(binary_message_chunks[j])
    extracted_message = extract_message_spatial(modified_image, chunk_size)
    extracted_messages.append(extracted_message)

# Combine all extracted chunks into the complete binary message
extracted_message_spatial = ''.join(extracted_messages)

# Write extracted binary message to a text file
extracted_message_path = os.path.join(output_dir, 'extracted_message.txt')
write_binary_message_to_file(extracted_message_spatial, extracted_message_path)
print(f"Extracted binary message saved to: {extracted_message_path}")

# Calculate SNR for each encoded image
snrs = []
for j in range(len(modified_images)):
    # Determine the current cover image index using modulo operation
    i = j % num_cover_images
    
    cover_image = cv2.imread(cover_image_files[i])
    cover_image_rgb = cv2.cvtColor(cover_image, cv2.COLOR_BGR2RGB)
    
    snr = calculate_snr(cover_image_rgb.astype(np.float64), modified_images[j].astype(np.float64))
    snrs.append(snr)
    print(f"SNR (spatial) for Image {j + 1}: {snr:.2f} dB")

# Calculate average SNR
average_snr = np.mean(snrs)
print(f"Average SNR (spatial) across all images: {average_snr:.2f} dB")

# Calculate maximum embedding capacity of the cover image
print(f"Maximum embedding capacity of a cover image: {max_capacity} bits")
